In [1]:
import os
import sys
print("Python version: ",sys.version)
print("Version info:   ",sys.version_info)

# add path to get to the modules renderneuron and parallelframe
sys.path.insert(0,'../src/')
import neuronswc as ns
import neuronmeshgenerator as nmg
import networkx as nx

Python version:  3.8.10 (default, Mar 13 2023, 10:26:41) 
[GCC 9.4.0]
Version info:    sys.version_info(major=3, minor=8, micro=10, releaselevel='final', serial=0)
JSON:            2.0.9
networkx:        3.1
scipy:           1.10.1


In [2]:
neuron_filename='../cells/0-2a.CNG.swc'
num_ref=8
start_dx=128.0
n_cir_points=12
sph_contours=32
sph_points=32
MESHFOLDER=nmg.make_meshes(neuron_filename,start_dx,num_ref,n_cir_points,sph_contours,sph_points)
nmg.write_vrn(MESHFOLDER)

Saved to  0-2a.CNG.mesh/0-2a.CNGrefinement0.swc
Saved to  0-2a.CNG.mesh/0-2a.CNG_1d_refinement0.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNG_3d_mesh_refinment0.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNGrefinement1.swc
Saved to  0-2a.CNG.mesh/0-2a.CNG_1d_refinement1.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNG_3d_mesh_refinment1.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNGrefinement2.swc
Saved to  0-2a.CNG.mesh/0-2a.CNG_1d_refinement2.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNG_3d_mesh_refinment2.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNGrefinement3.swc
Saved to  0-2a.CNG.mesh/0-2a.CNG_1d_refinement3.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNG_3d_mesh_refinment3.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNGrefinement4.swc
Saved to  0-2a.CNG.mesh/0-2a.CNG_1d_refinement4.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNG_3d_mesh_refinment4.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNGrefinement5.swc
Saved to  0-2a.CNG.mesh/0-2a.CNG_1d_refinement5.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNG_3d_mesh_refinment5.ugx
Saved to  0-2a.CNG.mesh/0-2a.CNGrefinement6.swc
Saved to  0-2a.C

In [ ]:
neuron_filename='../cells/228-21MG.CNG.swc'
G=ns.read_1d_neuron_swc(neuron_filename)
G=ns.remove_soma_line(G)
G1=ns.spline_neuron(G,1.0)

ns.write_1d_ugx(G1,'G1_oldlabel.ugx')
old_labels=list(set(G1.nodes()))
new_labels=list(nx.dfs_preorder_nodes(G1,source=1))

mapping=dict(zip(old_labels,new_labels))
G1=nx.relabel_nodes(G1,mapping)
ns.write_1d_ugx(G1,'G1_newlabel.ugx')
trunks,T=ns.get_trunks(G1)
ns.write_1d_ugx(T,'test_trunks.ugx')
ns.write_1d_ugx_soma_neurite(T,'soma_neurite.ugx')
nmg.write_mesh_with_soma_sphere_ugx(T,12,16,16,'trunks_mesh.ugx')

In [3]:
s='1 2 3 4 5 6 7'

In [4]:
s=s[:-1]

In [5]:
s

'1 2 3 4 5 6 '

In [6]:
s=s[::-1]

In [7]:
s

' 6 5 4 3 2 1'